Xóa những từ tiếng anh chuẩn có trong file ten_rieng.csv

In [7]:
import pandas as pd
from nltk.corpus import words

# 1. Cấu hình file
file_ten_rieng = 'ten_rieng.csv'          # File chứa danh sách tên riêng cần lọc
output_file = 'ten_rieng_cleaned.csv'     # File kết quả

try:
    print(f"Đang nạp từ điển NLTK từ: /Users/copy/nltk_data ...")

    # 2. Tạo bộ lọc từ điển (Set)
    # Chuyển tất cả về chữ thường để so sánh chính xác (boy == Boy)
    english_vocab = set(w.lower() for w in words.words())
    print(f"-> Đã nạp thành công {len(english_vocab)} từ tiếng Anh chuẩn.")

    # 3. Đọc file CSV tên riêng
    df_names = pd.read_csv(file_ten_rieng, encoding='utf-8-sig')

    # Tự động lấy tên cột đầu tiên (ví dụ: 'teen_code' hoặc 'word')
    col_name = df_names.columns[0]
    print(f"\nTổng số dòng ban đầu: {len(df_names)}")

    # 4. Định nghĩa hàm lọc
    def is_valid_name(name):
        # Chuyển từ cần kiểm tra về string, xóa khoảng trắng, về chữ thường
        clean_name = str(name).strip().lower()

        # Nếu từ này CÓ trong từ điển Anh -> Trả về False (để XÓA)
        # Nếu từ này KHÔNG có -> Trả về True (để GIỮ LẠI)
        return clean_name not in english_vocab

    # 5. Áp dụng bộ lọc
    # Giữ lại những dòng mà hàm is_valid_name trả về True
    df_cleaned = df_names[df_names[col_name].apply(is_valid_name)]

    # 6. Thống kê và Lưu file
    so_luong_xoa = len(df_names) - len(df_cleaned)
    df_cleaned.to_csv(output_file, index=False, encoding='utf-8-sig')

    print("-" * 40)
    print(f"Đã hoàn thành!")
    print(f"Số từ bị xóa (trùng tiếng Anh): {so_luong_xoa}")
    print(f"Số tên riêng còn lại: {len(df_cleaned)}")
    print(f"File sạch đã lưu tại: {output_file}")

    # In thử vài từ bị xóa để kiểm tra (nếu có)
    deleted = df_names[~df_names[col_name].apply(is_valid_name)][col_name].head(5).tolist()
    if deleted:
        print(f"Ví dụ các từ đã bị xóa: {deleted}")

except FileNotFoundError:
    print(f"Lỗi: Không tìm thấy file '{file_ten_rieng}' tại thư mục chạy code.")
except Exception as e:
    print(f"Lỗi: {e}")

Đang nạp từ điển NLTK từ: /Users/copy/nltk_data ...
-> Đã nạp thành công 234377 từ tiếng Anh chuẩn.

Tổng số dòng ban đầu: 9901
----------------------------------------
Đã hoàn thành!
Số từ bị xóa (trùng tiếng Anh): 697
Số tên riêng còn lại: 9204
File sạch đã lưu tại: ten_rieng_cleaned.csv
Ví dụ các từ đã bị xóa: ['Matra', 'Abraham', 'Ace', 'Adam', 'adam']


Xóa các từ có trong phien_am.csv ra khỏi ten_rieng.csv

In [8]:
import pandas as pd
import numpy as np

# ==============================================================================
# CẤU HÌNH FILE
# ==============================================================================
file_ten_rieng = 'ten_rieng_cleaned.csv'       # File tên riêng cần làm sạch
file_phien_am = 'phien_am.csv'         # File chứa các từ phiên âm (cần loại bỏ)
output_file = 'ten_rieng.csv'  # File kết quả

try:
    print("Đang đọc dữ liệu...")
    # 1. Đọc file
    # Dùng low_memory=False phòng trường hợp file lớn hoặc mix type
    df_names = pd.read_csv(file_ten_rieng, encoding='utf-8-sig')
    df_phien_am = pd.read_csv(file_phien_am, encoding='utf-8-sig')

    print(f"- File tên riêng: {df_names.shape[0]} dòng")
    print(f"- File phiên âm: {df_phien_am.shape[0]} dòng, {df_phien_am.shape[1]} cột")

    # 2. TẠO BLACKLIST TỪ FILE PHIÊN ÂM
    # Vì file phiên âm có nhiều cột, ta cần "ép phẳng" (flatten) tất cả về 1 danh sách duy nhất
    print("Đang xử lý dữ liệu phiên âm...")

    # Lấy toàn bộ dữ liệu, chuyển thành mảng 1 chiều, loại bỏ giá trị Null/NaN
    all_phien_am_values = df_phien_am.values.flatten()

    # Tạo set để tra cứu cho nhanh (convert về string chữ thường + xóa khoảng trắng thừa)
    # set() giúp loại bỏ các từ trùng lặp trong chính file phiên âm
    blacklist = set()
    for val in all_phien_am_values:
        if pd.notna(val): # Chỉ lấy nếu không phải là ô trống
            clean_val = str(val).strip().lower()
            if clean_val: # Nếu không phải chuỗi rỗng
                blacklist.add(clean_val)

    print(f"-> Đã tạo danh sách đen gồm {len(blacklist)} từ phiên âm độc nhất.")

    # 3. LỌC FILE TÊN RIÊNG
    col_name = df_names.columns[0] # Lấy cột đầu tiên của file tên riêng

    def is_valid_name(name):
        # Chuyển tên cần check về dạng chuẩn (thường, không space)
        clean_name = str(name).strip().lower()
        # Giữ lại nếu KHÔNG nằm trong blacklist
        return clean_name not in blacklist

    # Áp dụng bộ lọc
    df_clean = df_names[df_names[col_name].apply(is_valid_name)]

    # 4. THỐNG KÊ VÀ LƯU
    so_luong_bi_xoa = len(df_names) - len(df_clean)

    df_clean.to_csv(output_file, index=False, encoding='utf-8-sig')

    print("\n" + "="*30)
    print("HOÀN TẤT XỬ LÝ!")
    print(f"Số từ bị xóa (trùng phiên âm): {so_luong_bi_xoa}")
    print(f"Số tên riêng còn lại: {len(df_clean)}")
    print(f"File sạch đã lưu tại: {output_file}")

    # In thử vài ví dụ bị xóa để kiểm tra
    if so_luong_bi_xoa > 0:
        deleted_items = df_names[~df_names[col_name].apply(is_valid_name)][col_name].head(10).tolist()
        print(f"\nVí dụ các từ đã bị xóa: {deleted_items}")

except FileNotFoundError as e:
    print(f"Lỗi không tìm thấy file: {e}")
except Exception as e:
    print(f"Lỗi: {e}")

Đang đọc dữ liệu...
- File tên riêng: 9204 dòng
- File phiên âm: 41 dòng, 16 cột
Đang xử lý dữ liệu phiên âm...
-> Đã tạo danh sách đen gồm 188 từ phiên âm độc nhất.

HOÀN TẤT XỬ LÝ!
Số từ bị xóa (trùng phiên âm): 23
Số tên riêng còn lại: 9181
File sạch đã lưu tại: ten_rieng_final.csv

Ví dụ các từ đã bị xóa: ['A Di Đà', 'A di đà', 'Apsara', 'apsara', 'Cờ-rút-sì', 'Élysées', 'la mar de suenos', 'maxcơva', 'Métro', 'Mitsipsipi']


Xóa ten_rieng_cleaned.csv khỏi english.csv

In [9]:
import pandas as pd

# ==============================================================================
# CẤU HÌNH FILE
# ==============================================================================
file_english = 'english.csv'                # File gốc (Từ điển tiếng Anh)
file_remove = 'ten_rieng.csv'  # File chứa các từ cần xóa (Tên riêng)
output_file = 'english_cleaned.csv'           # File kết quả sau khi lọc

try:
    print("Đang đọc dữ liệu...")
    # 1. Đọc file
    df_eng = pd.read_csv(file_english, encoding='utf-8-sig')
    df_rem = pd.read_csv(file_remove, encoding='utf-8-sig')

    print(f"- File English gốc: {len(df_eng)} dòng")
    print(f"- File Tên riêng (để xóa): {len(df_rem)} dòng")

    # 2. Tạo danh sách các từ cần xóa (Blacklist)
    # Lấy cột đầu tiên của file ten_rieng, đưa về chữ thường để so sánh
    col_rem = df_rem.columns[0]
    remove_set = set(df_rem[col_rem].astype(str).str.strip().str.lower())

    # 3. Thực hiện lọc
    col_eng = df_eng.columns[0] # Lấy cột đầu tiên của file english

    # Logic: Giữ lại những từ trong english.csv KHÔNG nằm trong remove_set
    # Dùng ~ (NOT) và isin
    mask = ~df_eng[col_eng].astype(str).str.strip().str.lower().isin(remove_set)
    df_final = df_eng[mask]

    # 4. Thống kê và lưu file
    so_luong_xoa = len(df_eng) - len(df_final)

    df_final.to_csv(output_file, index=False, encoding='utf-8-sig')

    print("\n" + "="*30)
    print("HOÀN TẤT!")
    print(f"Số từ tiếng Anh bị xóa (do trùng tên riêng): {so_luong_xoa}")
    print(f"Số từ tiếng Anh còn lại: {len(df_final)}")
    print(f"File kết quả lưu tại: {output_file}")

    # In thử ví dụ
    if so_luong_xoa > 0:
        deleted = df_eng[~mask][col_eng].head(5).tolist()
        print(f"\nVí dụ các từ đã bị xóa: {deleted}")

except FileNotFoundError as e:
    print(f"Lỗi: Không tìm thấy file. {e}")
except Exception as e:
    print(f"Lỗi: {e}")

Đang đọc dữ liệu...
- File English gốc: 14125 dòng
- File Tên riêng (để xóa): 9181 dòng

HOÀN TẤT!
Số từ tiếng Anh bị xóa (do trùng tên riêng): 631
Số từ tiếng Anh còn lại: 13494
File kết quả lưu tại: english_cleaned.csv

Ví dụ các từ đã bị xóa: ['50 Cent', '52hz', '5days', '88rising', 'A1']


Xóa phien_am.csv khỏi english_final.csv

In [11]:
import pandas as pd

# ==============================================================================
# CẤU HÌNH FILE
# ==============================================================================
file_english = 'english_cleaned.csv'      # File tiếng Anh (đã lọc tên riêng ở bước trước)
file_phien_am = 'phien_am.csv'          # File phiên âm (chứa nhiều cột)
output_file = 'english_really_final.csv' # File kết quả cuối cùng

try:
    print("Đang đọc dữ liệu...")
    # 1. Đọc file
    df_eng = pd.read_csv(file_english, encoding='utf-8-sig')
    df_phien_am = pd.read_csv(file_phien_am, encoding='utf-8-sig')

    print(f"- File English hiện tại: {len(df_eng)} dòng")
    print(f"- File Phiên âm: {df_phien_am.shape[0]} dòng, {df_phien_am.shape[1]} cột")

    # 2. Xử lý file Phiên âm -> Tạo Blacklist
    # Gom tất cả các cột lại thành 1 danh sách duy nhất
    print("Đang tạo danh sách đen từ file phiên âm...")
    all_phien_am_values = df_phien_am.values.flatten()

    # Tạo set blacklist (chuẩn hóa về chữ thường để so sánh)
    blacklist = set()
    for val in all_phien_am_values:
        if pd.notna(val): # Bỏ qua giá trị rỗng
            clean_val = str(val).strip().lower()
            if clean_val:
                blacklist.add(clean_val)

    print(f"-> Blacklist phiên âm có {len(blacklist)} từ.")

    # 3. Lọc file English
    col_eng = df_eng.columns[0] # Lấy cột chứa từ tiếng Anh

    # Logic: Giữ lại từ nếu nó KHÔNG nằm trong blacklist
    # (So sánh ở dạng lowercase)
    mask = ~df_eng[col_eng].astype(str).str.strip().str.lower().isin(blacklist)
    df_final = df_eng[mask]

    # 4. Lưu kết quả
    so_luong_xoa = len(df_eng) - len(df_final)

    df_final.to_csv(output_file, index=False, encoding='utf-8-sig')

    print("\n" + "="*30)
    print("HOÀN TẤT XỬ LÝ!")
    print(f"Số từ tiếng Anh bị xóa (do trùng phiên âm): {so_luong_xoa}")
    print(f"Số lượng từ tiếng Anh cuối cùng: {len(df_final)}")
    print(f"File đã lưu tại: {output_file}")

    # In ví dụ các từ bị xóa
    if so_luong_xoa > 0:
        deleted = df_eng[~mask][col_eng].head(10).tolist()
        print(f"\nVí dụ các từ bị xóa khỏi từ điển Anh: {deleted}")

except FileNotFoundError as e:
    print(f"Lỗi: Không tìm thấy file. {e}")
except Exception as e:
    print(f"Lỗi: {e}")

Đang đọc dữ liệu...
- File English hiện tại: 13494 dòng
- File Phiên âm: 41 dòng, 16 cột
Đang tạo danh sách đen từ file phiên âm...
-> Blacklist phiên âm có 188 từ.

HOÀN TẤT XỬ LÝ!
Số từ tiếng Anh bị xóa (do trùng phiên âm): 18
Số lượng từ tiếng Anh cuối cùng: 13476
File đã lưu tại: english_really_final.csv

Ví dụ các từ bị xóa khỏi từ điển Anh: ['alleluia', 'baby', 'Bioneun georieseo', 'ciao', 'ghita', 'Je taimais', 'kimono', 'Konnichiwa', 'Namo', 'oppa']


Xóa english_really_final.csv khỏi phien_am.csv

In [13]:
import pandas as pd
from nltk.corpus import words

# 1. Cấu hình file
file_ten_rieng = 'phien_am.csv'          # File chứa danh sách tên riêng cần lọc
output_file = 'phien_am_cleaned.csv'     # File kết quả

try:
    print(f"Đang nạp từ điển NLTK từ: /Users/copy/nltk_data ...")

    # 2. Tạo bộ lọc từ điển (Set)
    # Chuyển tất cả về chữ thường để so sánh chính xác (boy == Boy)
    english_vocab = set(w.lower() for w in words.words())
    print(f"-> Đã nạp thành công {len(english_vocab)} từ tiếng Anh chuẩn.")

    # 3. Đọc file CSV tên riêng
    df_names = pd.read_csv(file_ten_rieng, encoding='utf-8-sig')

    # Tự động lấy tên cột đầu tiên (ví dụ: 'teen_code' hoặc 'word')
    col_name = df_names.columns[0]
    print(f"\nTổng số dòng ban đầu: {len(df_names)}")

    # 4. Định nghĩa hàm lọc
    def is_valid_name(name):
        # Chuyển từ cần kiểm tra về string, xóa khoảng trắng, về chữ thường
        clean_name = str(name).strip().lower()

        # Nếu từ này CÓ trong từ điển Anh -> Trả về False (để XÓA)
        # Nếu từ này KHÔNG có -> Trả về True (để GIỮ LẠI)
        return clean_name not in english_vocab

    # 5. Áp dụng bộ lọc
    # Giữ lại những dòng mà hàm is_valid_name trả về True
    df_cleaned = df_names[df_names[col_name].apply(is_valid_name)]

    # 6. Thống kê và Lưu file
    so_luong_xoa = len(df_names) - len(df_cleaned)
    df_cleaned.to_csv(output_file, index=False, encoding='utf-8-sig')

    print("-" * 40)
    print(f"Đã hoàn thành!")
    print(f"Số từ bị xóa (trùng tiếng Anh): {so_luong_xoa}")
    print(f"Số tên riêng còn lại: {len(df_cleaned)}")
    print(f"File sạch đã lưu tại: {output_file}")

    # In thử vài từ bị xóa để kiểm tra (nếu có)
    deleted = df_names[~df_names[col_name].apply(is_valid_name)][col_name].head(5).tolist()
    if deleted:
        print(f"Ví dụ các từ đã bị xóa: {deleted}")

except FileNotFoundError:
    print(f"Lỗi: Không tìm thấy file '{file_ten_rieng}' tại thư mục chạy code.")
except Exception as e:
    print(f"Lỗi: {e}")

Đang nạp từ điển NLTK từ: /Users/copy/nltk_data ...
-> Đã nạp thành công 234377 từ tiếng Anh chuẩn.

Tổng số dòng ban đầu: 41
----------------------------------------
Đã hoàn thành!
Số từ bị xóa (trùng tiếng Anh): 40
Số tên riêng còn lại: 1
File sạch đã lưu tại: phien_am_cleaned.csv
Ví dụ các từ đã bị xóa: [nan, nan, nan, nan, nan]


In [16]:
import pandas as pd

# ==============================================================================
# CẤU HÌNH INPUT (Tên file kết quả cuối cùng của bạn)
# ==============================================================================
file_english = 'english.csv'
file_phien_am = 'phien_am.csv'
file_ten_rieng = 'ten_rieng.csv'

try:
    print("Đang tải 3 file để đối chiếu chéo...")

    # 1. Đọc dữ liệu
    df_eng = pd.read_csv(file_english, encoding='utf-8-sig')
    df_pa = pd.read_csv(file_phien_am, encoding='utf-8-sig')
    df_tr = pd.read_csv(file_ten_rieng, encoding='utf-8-sig')

    # 2. Tạo 3 tập hợp (SET) chuẩn hóa
    # Hàm helper để chuẩn hóa (lowercase, strip, bỏ nan)
    def get_clean_set(df_input):
        # Nếu là dataframe nhiều cột (như phien_am), flatten trước
        values = df_input.values.flatten()
        clean_set = set()
        for val in values:
            if pd.notna(val):
                s = str(val).strip().lower()
                if s: clean_set.add(s)
        return clean_set

    set_eng = get_clean_set(df_eng)
    set_pa = get_clean_set(df_pa)
    set_tr = get_clean_set(df_tr)

    print(f"-> Số lượng từ: English={len(set_eng)}, Phiên Âm={len(set_pa)}, Tên Riêng={len(set_tr)}")

    # 3. Tìm điểm trùng lặp (Intersection)
    # A & B trả về các phần tử chung của A và B
    Trung_Eng_vs_PA = set_eng & set_pa
    Trung_Eng_vs_TR = set_eng & set_tr
    Trung_PA_vs_TR  = set_pa  & set_tr

    # 4. Báo cáo kết quả
    print("\n" + "="*40)
    print("KẾT QUẢ KIỂM TRA ĐỘC LẬP (DISJOINT CHECK)")
    print("="*40)

    # Check 1: English vs Phiên Âm
    if len(Trung_Eng_vs_PA) == 0:
        print("✅ English và Phiên Âm: SẠCH (Không trùng).")
    else:
        print(f"❌ CẢNH BÁO: Còn {len(Trung_Eng_vs_PA)} từ trùng giữa English và Phiên Âm.")
        print(f"   Ví dụ: {list(Trung_Eng_vs_PA)[:5]}")

    # Check 2: English vs Tên Riêng
    if len(Trung_Eng_vs_TR) == 0:
        print("✅ English và Tên Riêng: SẠCH (Không trùng).")
    else:
        print(f"❌ CẢNH BÁO: Còn {len(Trung_Eng_vs_TR)} từ trùng giữa English và Tên Riêng.")
        print(f"   Ví dụ: {list(Trung_Eng_vs_TR)[:5]}")

    # Check 3: Phiên Âm vs Tên Riêng
    if len(Trung_PA_vs_TR) == 0:
        print("✅ Phiên Âm và Tên Riêng: SẠCH (Không trùng).")
    else:
        print(f"❌ CẢNH BÁO: Còn {len(Trung_PA_vs_TR)} từ trùng giữa Phiên Âm và Tên Riêng.")
        print(f"   Ví dụ: {list(Trung_PA_vs_TR)[:5]}")

except Exception as e:
    print(f"Lỗi: {e}")

Đang tải 3 file để đối chiếu chéo...
-> Số lượng từ: English=9964, Phiên Âm=188, Tên Riêng=8423

KẾT QUẢ KIỂM TRA ĐỘC LẬP (DISJOINT CHECK)
✅ English và Phiên Âm: SẠCH (Không trùng).
✅ English và Tên Riêng: SẠCH (Không trùng).
✅ Phiên Âm và Tên Riêng: SẠCH (Không trùng).


xóa các từ trong ntlk chuẩn khỏi english_really_final.csv

In [15]:
import pandas as pd
from nltk.corpus import words

# 1. Cấu hình file
file_ten_rieng = 'english_really_final.csv'          # File chứa danh sách tên riêng cần lọc
output_file = 'english.csv'  # File kết quả

try:
    print(f"Đang nạp từ điển NLTK từ: /Users/copy/nltk_data ...")

    # 2. Tạo bộ lọc từ điển (Set)
    # Chuyển tất cả về chữ thường để so sánh chính xác (boy == Boy)
    english_vocab = set(w.lower() for w in words.words())
    print(f"-> Đã nạp thành công {len(english_vocab)} từ tiếng Anh chuẩn.")

    # 3. Đọc file CSV tên riêng
    df_names = pd.read_csv(file_ten_rieng, encoding='utf-8-sig')

    # Tự động lấy tên cột đầu tiên (ví dụ: 'teen_code' hoặc 'word')
    col_name = df_names.columns[0]
    print(f"\nTổng số dòng ban đầu: {len(df_names)}")

    # 4. Định nghĩa hàm lọc
    def is_valid_name(name):
        # Chuyển từ cần kiểm tra về string, xóa khoảng trắng, về chữ thường
        clean_name = str(name).strip().lower()

        # Nếu từ này CÓ trong từ điển Anh -> Trả về False (để XÓA)
        # Nếu từ này KHÔNG có -> Trả về True (để GIỮ LẠI)
        return clean_name not in english_vocab

    # 5. Áp dụng bộ lọc
    # Giữ lại những dòng mà hàm is_valid_name trả về True
    df_cleaned = df_names[df_names[col_name].apply(is_valid_name)]

    # 6. Thống kê và Lưu file
    so_luong_xoa = len(df_names) - len(df_cleaned)
    df_cleaned.to_csv(output_file, index=False, encoding='utf-8-sig')

    print("-" * 40)
    print(f"Đã hoàn thành!")
    print(f"Số từ bị xóa (trùng tiếng Anh): {so_luong_xoa}")
    print(f"Số tên riêng còn lại: {len(df_cleaned)}")
    print(f"File sạch đã lưu tại: {output_file}")

    # In thử vài từ bị xóa để kiểm tra (nếu có)
    deleted = df_names[~df_names[col_name].apply(is_valid_name)][col_name].head(5).tolist()
    if deleted:
        print(f"Ví dụ các từ đã bị xóa: {deleted}")

except FileNotFoundError:
    print(f"Lỗi: Không tìm thấy file '{file_ten_rieng}' tại thư mục chạy code.")
except Exception as e:
    print(f"Lỗi: {e}")

Đang nạp từ điển NLTK từ: /Users/copy/nltk_data ...
-> Đã nạp thành công 234377 từ tiếng Anh chuẩn.

Tổng số dòng ban đầu: 13476
----------------------------------------
Đã hoàn thành!
Số từ bị xóa (trùng tiếng Anh): 3512
Số tên riêng còn lại: 9964
File sạch đã lưu tại: english_out_standard.csv
Ví dụ các từ đã bị xóa: ['a', 'aam', 'abandon', 'abandoned', 'ability']


Dùng thư viện từ tiếng việt chuẩn nlp , loại những từ có trong thư viện ra khỏi han_viet.csv

In [21]:
import pandas as pd
import requests

# ==============================================================================
# CẤU HÌNH
# ==============================================================================
file_input = 'hanviet_dictionary.csv'             # File đầu vào
output_file = 'han_viet_filtered.csv'   # File kết quả
target_source = 'phongp-hanviet-pinyin' # Nguồn cần kiểm tra để lọc

# Link từ điển tiếng Việt chuẩn (Viet74K)
URL_VIETNAMESE_DICT = "https://raw.githubusercontent.com/duyet/vietnamese-wordlist/master/Viet74K.txt"

try:
    # 1. Tải từ điển Tiếng Việt chuẩn
    print("Đang tải từ điển Tiếng Việt chuẩn...")
    response = requests.get(URL_VIETNAMESE_DICT)

    if response.status_code == 200:
        vietnamese_vocab = set(
            line.strip().lower()
            for line in response.text.splitlines()
            if line.strip()
        )
        print(f"-> Đã nạp {len(vietnamese_vocab)} từ vựng chuẩn.")
    else:
        raise Exception("Lỗi tải từ điển.")

    # 2. Đọc file CSV
    print(f"\nĐang xử lý file '{file_input}'...")
    df = pd.read_csv(file_input, encoding='utf-8-sig')

    # Kiểm tra xem file có cột 'source' không
    if 'source' not in df.columns:
        raise Exception("File CSV không có cột 'source'. Vui lòng kiểm tra lại.")

    # Tên cột chứa từ ngữ (giả sử là cột đầu tiên nếu không phải tên là 'hanviet')
    col_word = 'hanviet' if 'hanviet' in df.columns else df.columns[0]

    print(f"-> Tổng số dòng ban đầu: {len(df)}")

    # 3. Hàm logic lọc (CORE LOGIC)
    def should_keep_row(row):
        word = str(row[col_word]).strip().lower()
        source = str(row['source']).strip()

        # LOGIC 1: Nếu source KHÁC 'phongp-hanviet-pinyin' -> Luôn giữ lại (True)
        if source != target_source:
            return True

        # LOGIC 2: Nếu source LÀ 'phongp-hanviet-pinyin'
        # Chỉ giữ lại nếu từ này KHÔNG nằm trong từ điển phổ thông
        # (Nghĩa là xóa nếu nó là từ phổ thông)
        is_common_word = word in vietnamese_vocab
        if is_common_word:
            return False # XÓA
        else:
            return True  # GIỮ (Vì là từ hiếm/lạ)

    # 4. Áp dụng
    # axis=1 nghĩa là quét theo từng hàng
    mask = df.apply(should_keep_row, axis=1)
    df_filtered = df[mask]

    # 5. Thống kê và Lưu
    so_luong_xoa = len(df) - len(df_filtered)

    df_filtered.to_csv(output_file, index=False, encoding='utf-8-sig')

    print("\n" + "="*40)
    print(f"ĐÃ HOÀN TẤT!")
    print(f"Số từ bị xóa: {so_luong_xoa}")
    print(f"(Chỉ xóa các từ thuộc source '{target_source}' và trùng với từ điển phổ thông)")
    print(f"Số dòng còn lại: {len(df_filtered)}")
    print(f"File kết quả: {output_file}")

    # In ví dụ kiểm chứng
    if so_luong_xoa > 0:
        print("\nVí dụ các dòng bị xóa:")
        deleted_rows = df[~mask].head(5)
        print(deleted_rows[[col_word, 'source']])

except Exception as e:
    print(f"Lỗi: {e}")

Đang tải từ điển Tiếng Việt chuẩn...
-> Đã nạp 72535 từ vựng chuẩn.

Đang xử lý file 'hanviet_dictionary.csv'...
-> Tổng số dòng ban đầu: 16631

ĐÃ HOÀN TẤT!
Số từ bị xóa: 10223
(Chỉ xóa các từ thuộc source 'phongp-hanviet-pinyin' và trùng với từ điển phổ thông)
Số dòng còn lại: 6408
File kết quả: han_viet_filtered.csv

Ví dụ các dòng bị xóa:
     hanviet                 source
5101  thướng  phongp-hanviet-pinyin
5102  thượng  phongp-hanviet-pinyin
5103      cơ  phongp-hanviet-pinyin
5104      kỳ  phongp-hanviet-pinyin
5105   trung  phongp-hanviet-pinyin


Lọc từ tiếng việt chuẩn ra khỏi ten_rieng.csv

In [22]:
import pandas as pd
import requests

# ==============================================================================
# CẤU HÌNH
# ==============================================================================
file_input = 'ten_rieng.csv'             # File đầu vào
output_file = 'ten_rieng_no_common.csv'  # File kết quả

# Link từ điển tiếng Việt chuẩn
URL_VIETNAMESE_DICT = "https://raw.githubusercontent.com/duyet/vietnamese-wordlist/master/Viet74K.txt"

try:
    print("1. Đang tải từ điển Tiếng Việt chuẩn...")
    response = requests.get(URL_VIETNAMESE_DICT)

    if response.status_code == 200:
        # Nạp từ điển vào Set
        vietnamese_vocab = set(
            line.strip().lower()
            for line in response.text.splitlines()
            if line.strip()
        )
        print(f"-> Đã nạp thành công: {len(vietnamese_vocab)} từ vựng chuẩn.")
    else:
        raise Exception("Không tải được từ điển online.")

    print(f"\n2. Đang đọc file '{file_input}'...")
    df = pd.read_csv(file_input, encoding='utf-8-sig')
    col_name = df.columns[0]

    print(f"-> Tổng số dòng ban đầu: {len(df)}")

    # 3. Định nghĩa hàm XỬ LÝ KÉP (Vừa lọc, vừa phân loại)
    def process_name(row):
        name = str(row[col_name]).strip()
        name_lower = name.lower()

        # BƯỚC A: Kiểm tra xem có phải từ phổ thông không
        if name_lower in vietnamese_vocab:
            return "DELETE" # Đánh dấu để xóa

        # BƯỚC B: Nếu không phải từ phổ thông -> Phân loại
        if ' ' in name: # Nếu có khoảng trắng -> Là cụm từ
            return "phrase"
        else:           # Không có khoảng trắng -> Là từ đơn
            return "word"

    # 4. Áp dụng logic
    # Tạo một cột tạm thời 'type' để lưu kết quả phân loại
    df['type'] = df.apply(process_name, axis=1)

    # 5. Tách dữ liệu
    # Chỉ lấy những dòng không bị đánh dấu DELETE
    df_clean = df[df['type'] != "DELETE"].copy()

    # Tính toán số lượng đã xóa
    so_luong_xoa = len(df) - len(df_clean)

    # 6. Lưu file (Kèm theo cột type để bạn dễ quản lý)
    # File sẽ có dạng: teen_code | type (word/phrase)
    df_clean.to_csv(output_file, index=False, encoding='utf-8-sig')

    print("\n" + "="*40)
    print("HOÀN TẤT!")
    print(f"Số lượng bị xóa (trùng từ điển): {so_luong_xoa}")
    print(f"Số tên riêng còn lại: {len(df_clean)}")

    # Thống kê chi tiết
    count_word = len(df_clean[df_clean['type'] == 'word'])
    count_phrase = len(df_clean[df_clean['type'] == 'phrase'])
    print(f"-> Trong đó: {count_word} từ đơn (word) và {count_phrase} cụm từ (phrase)")

    print(f"File kết quả: {output_file}")

    # Ví dụ bị xóa
    if so_luong_xoa > 0:
        print("\nVí dụ các từ bị xóa (Common words):")
        print(df[df['type'] == "DELETE"][col_name].head(5).tolist())

except Exception as e:
    print(f"Lỗi: {e}")

1. Đang tải từ điển Tiếng Việt chuẩn...
-> Đã nạp thành công: 72535 từ vựng chuẩn.

2. Đang đọc file 'ten_rieng.csv'...
-> Tổng số dòng ban đầu: 9181

HOÀN TẤT!
Số lượng bị xóa (trùng từ điển): 1786
Số tên riêng còn lại: 7395
-> Trong đó: 1920 từ đơn (word) và 5475 cụm từ (phrase)
File kết quả: ten_rieng_no_common.csv

Ví dụ các từ bị xóa (Common words):
['Trưng', 'Á Châu', 'A Di Đà Phật', 'Á Đông', 'Ái']
